In [93]:
import pandas as pd
import boto3
import json
import configparser
import psycopg2
import create_tables

%load_ext autoreload
%autoreload 2
from sql_queries import *

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [94]:
""" Get the parmas of created redshift cluster """

config = configparser.ConfigParser()
config.read_file(open('dwh.cfg'))
KEY=config.get('AWS','key')
SECRET= config.get('AWS','secret')

DB_NAME= config.get("CLUSTER","DB_NAME")
DB_USER= config.get("CLUSTER","DB_USER")
DB_PASSWORD= config.get("CLUSTER","DB_PASSWORD")
DB_PORT = config.get("CLUSTER","DB_PORT")

HOST = config.get("CLUSTER","HOST")
ROLE_ARN = config.get("IAM_ROLE","ARN")

In [95]:
""" Connect to the Redshift Cluster"""
conn = psycopg2.connect("host={} dbname={} user={} password={} port={}".format(*config['CLUSTER'].values()))


In [96]:
cur = conn.cursor()

In [13]:
s3 = boto3.resource('s3',
                       region_name="us-west-2",
                       aws_access_key_id=KEY,
                       aws_secret_access_key=SECRET
                   )

sampleDbBucket =  s3.Bucket("udacity-dend")

# TODO: Iterate over bucket objects starting 

counter = 0
for obj in sampleDbBucket.objects.filter(Prefix="song"):
    counter += 1
    print(obj)
    
    if counter == 10:
        break

s3.ObjectSummary(bucket_name='udacity-dend', key='song-data/')
s3.ObjectSummary(bucket_name='udacity-dend', key='song-data/A/A/A/TRAAAAK128F9318786.json')
s3.ObjectSummary(bucket_name='udacity-dend', key='song-data/A/A/A/TRAAAAV128F421A322.json')
s3.ObjectSummary(bucket_name='udacity-dend', key='song-data/A/A/A/TRAAABD128F429CF47.json')
s3.ObjectSummary(bucket_name='udacity-dend', key='song-data/A/A/A/TRAAACN128F9355673.json')
s3.ObjectSummary(bucket_name='udacity-dend', key='song-data/A/A/A/TRAAAEA128F935A30D.json')
s3.ObjectSummary(bucket_name='udacity-dend', key='song-data/A/A/A/TRAAAED128E0783FAB.json')
s3.ObjectSummary(bucket_name='udacity-dend', key='song-data/A/A/A/TRAAAEM128F93347B9.json')
s3.ObjectSummary(bucket_name='udacity-dend', key='song-data/A/A/A/TRAAAEW128F42930C0.json')
s3.ObjectSummary(bucket_name='udacity-dend', key='song-data/A/A/A/TRAAAFD128F92F423A.json')


# Creating staging_songs table

In [8]:
# drop table
cur.execute(staging_songs_table_drop)
conn.commit()

# create table
cur.execute(staging_songs_table_create)
conn.commit()

In [22]:
cur.execute("select * from staging_songs")
print(cur.fetchall())

[]


In [29]:
# load data into clusters
cur.execute(staging_songs_copy)
conn.commit()

In [53]:
cur.execute("select * from staging_songs limit 10")
for data in cur.fetchall():
    print(data)
    
cur.execute("select COUNT(*) from staging_songs")
for data in cur.fetchall():
    print("\n", data)

(1, 'ARIG6O41187B988BDD', 37.16793, -95.84502, 'United States', 'Richard Souther', 'SOUQQEA12A8C134B1B', 'High Tide', 228.5971, 0)
(1, 'AR6468X1187FB5AA0C', None, None, '', 'Micky Modelle', 'SOFXNXU12AB018A46E', 'Always On My Mind', 280.94649, 0)
(1, 'AR3DXTG1187FB38776', None, None, 'Bexleyheath, Kent, England', 'Kate Bush', 'SOGYILS12AF72A82AD', 'Coffee Homeground', 219.0624, 1978)
(1, 'ARQY5EG1187FB57063', None, None, '', 'Chuck Loeb', 'SOJUCAI12A8C135D62', 'New Life', 325.25016, 0)
(1, 'ARJIUJH1187B9B84FD', 34.94652, -89.4373, 'Slayden, MS', 'Charlie Feathers', 'SOUUERM12AB01850E4', 'Frankie & Johnny', 167.96689, 0)
(1, 'AR0TKGM1187B98B40E', 51.50632, -0.12714, 'London', 'Stereolab', 'SOKIODI12AB01839D3', 'The Nth Degree', 253.51791, 2008)
(1, 'AR7UGOA1187B9B2AD0', None, None, '', 'Malevolent Creation', 'SOVDHCR12A6701F1B3', 'Millions', 146.28526, 1997)
(1, 'ARW3OU61187B98A81E', None, None, 'London, England', 'Hexstatic', 'SOUMUUK12AB018AEF6', 'Intermission', 99.36934, 0)
(1, 'ARRE

# Creating staging_events table

In [39]:
# drop table
cur.execute(staging_events_table_drop)
conn.commit()

# create table
cur.execute(staging_events_table_create)
conn.commit()

In [40]:
cur.execute("select * from staging_events")
print(cur.fetchall())

[]


In [47]:
# load data into clusters
cur.execute(staging_events_copy)
conn.commit()

In [52]:
cur.execute("select * from staging_events limit 10")
for data in cur.fetchall():
    print(data)
    
cur.execute("select COUNT(*) from staging_events")
for data in cur.fetchall():
    print("\n",data)

(None, 'Logged Out', None, None, 0, None, None, 'free', None, 'PUT', 'Login', None, 52, None, 307, '1541207073796', None, None)
(None, 'Logged In', 'Celeste', 'F', 1, 'Williams', None, 'free', 'Klamath Falls, OR', 'GET', 'Home', 1541077528796.0, 52, None, 200, '1541207123796', '"Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/37.0.2062.103 Safari/537.36"', 53)
('Mynt', 'Logged In', 'Celeste', 'F', 2, 'Williams', 166.94812, 'free', 'Klamath Falls, OR', 'PUT', 'NextSong', 1541077528796.0, 52, 'Playa Haters', 200, '1541207150796', '"Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/37.0.2062.103 Safari/537.36"', 53)
('Taylor Swift', 'Logged In', 'Celeste', 'F', 3, 'Williams', 230.47791, 'free', 'Klamath Falls, OR', 'PUT', 'NextSong', 1541077528796.0, 52, 'You Belong With Me', 200, '1541207316796', '"Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/37.0.2062.103 Safari/537.36"', 53)
('Amy Win

## Songs Table

In [54]:
# drop table
cur.execute(song_table_drop)
conn.commit()

# create table
cur.execute(song_table_create)
conn.commit()

In [62]:
# load data into clusters
cur.execute(song_table_insert)
conn.commit()

In [66]:
cur.execute("select * from songs limit 10")
for data in cur.fetchall():
    print(data)
    
cur.execute("select COUNT(*) from songs")
for data in cur.fetchall():
    print("\n",data)

('SOJUCAI12A8C135D62', 'New Life', 'ARQY5EG1187FB57063', 0, 325.25016)
('SOELRRQ12AB0188C34', "Let's Go", 'AR20CFC1187B98A25D', 0, 160.93995)
('SOOUQDN12AB018E452', 'Blackout Summer', 'ARXZXNC122BCFCB7B5', 2009, 168.25424)
('SONYRLV12A67ADE583', 'Ultrasónica', 'ARY5U6T1187FB5AE47', 2001, 524.17261)
('SOEBDPE12A8C13C0CA', 'God Drinks At The Sando', 'ARM6U4Q1187B99ACBB', 1999, 172.82567)
('SOOKXKB12A8C1390B5', 'Go Home', 'AR1OGXT1187B9893EB', 1994, 194.16771)
('SOLWWCO12A58A77015', 'A Little Tear', 'AROUOZZ1187B9ABE51', 0, 416.44363)
('SOOMNEP12A8C140050', 'The First Noel', 'AR7OA5G1187B98D3A0', 2006, 52.63628)
('SOXAOIN12A8C138D34', "That Rock Won't Roll", 'ARR5CEB1187FB5A3FF', 1986, 210.46812)
('SOJPMQE12AAF3B4313', 'What Truth?', 'ARYZ9FU1187B99123F', 1998, 124.13342)

 (14896,)


## Artists Table

In [70]:
# drop table
cur.execute(artist_table_drop)
conn.commit()

# create table
cur.execute(artist_table_create)
conn.commit()

# load data into clusters
cur.execute(artist_table_insert)
conn.commit()

In [71]:
cur.execute("select * from artists limit 10")
for data in cur.fetchall():
    print(data)
    
cur.execute("select COUNT(*) from artists")
for data in cur.fetchall():
    print("\n",data)

('AR73AIO1187B9AD57B', 'Western Addiction', 'San Francisco, CA', 37.77916, -122.42005)
('ARZPDAD1187B98D940', 'Fattburger', '', None, None)
('AROO15N1187B99B64C', 'Stunt Rock', '', None, None)
('AR3PTSL1187FB5912E', 'Sielun Veljet', '', None, None)
('ARNVVXC1187FB392EE', 'Poisonblack', '', None, None)
('ART0ATJ1187FB531D2', 'Dar Williams', 'Mount Kisco, NY', 41.20633, -73.72684)
('AR3UF0G119B340300E', 'Xymox', '', None, None)
('ARH39ZK1187B9ABC22', 'Girl Talk', 'California - SF', None, None)
('ARAW35N1187B994CCA', 'Queensryche', '', None, None)
('ARPUEAL1187FB57814', "Gov't Mule", '', None, None)

 (10025,)


## Users Table


In [83]:
# drop table
cur.execute(user_table_drop)
conn.commit()

# create table
cur.execute(user_table_create)
conn.commit()

# load data into clusters
cur.execute(user_table_insert)
conn.commit()

In [84]:
cur.execute("select * from users limit 10")
for data in cur.fetchall():
    print(data)
    
cur.execute("select COUNT(*) from users")
for data in cur.fetchall():
    print("\n",data)

(101, 'Jayden', 'Fox', 'M', 'free')
(17, 'Makinley', 'Jones', 'F', 'free')
(61, 'Samuel', 'Gonzalez', 'M', 'free')
(23, 'Morris', 'Gilmore', 'M', 'free')
(6, 'Cecilia', 'Owens', 'F', 'free')
(28, 'Brantley', 'West', 'M', 'free')
(92, 'Ryann', 'Smith', 'F', 'free')
(71, 'Ayleen', 'Wise', 'F', 'free')
(67, 'Colm', 'Santana', 'M', 'free')
(20, 'Aiden', 'Ramirez', 'M', 'paid')

 (104,)


## time table

In [89]:
# drop table
cur.execute(time_table_drop)
conn.commit()

# create table
cur.execute(time_table_create)
conn.commit()

# load data into clusters
cur.execute(time_table_insert)
conn.commit()

In [90]:
cur.execute("select * from time limit 10")
for data in cur.fetchall():
    print(data)
    
cur.execute("select COUNT(*) from time")
for data in cur.fetchall():
    print("\n",data)

(datetime.datetime(2018, 11, 3, 14, 17, 50, 796000), 14, 3, 44, 11, 2018, 6)
(datetime.datetime(2018, 11, 3, 16, 10, 32, 796000), 16, 3, 44, 11, 2018, 6)
(datetime.datetime(2018, 11, 3, 16, 45, 1, 796000), 16, 3, 44, 11, 2018, 6)
(datetime.datetime(2018, 11, 3, 17, 2, 52, 796000), 17, 3, 44, 11, 2018, 6)
(datetime.datetime(2018, 11, 3, 17, 19, 54, 796000), 17, 3, 44, 11, 2018, 6)
(datetime.datetime(2018, 11, 3, 17, 34, 42, 796000), 17, 3, 44, 11, 2018, 6)
(datetime.datetime(2018, 11, 3, 17, 47, 24, 796000), 17, 3, 44, 11, 2018, 6)
(datetime.datetime(2018, 11, 3, 18, 3, 23, 796000), 18, 3, 44, 11, 2018, 6)
(datetime.datetime(2018, 11, 2, 2, 42, 48, 796000), 2, 2, 44, 11, 2018, 5)
(datetime.datetime(2018, 11, 2, 9, 12, 32, 796000), 9, 2, 44, 11, 2018, 5)

 (6813,)


In [98]:
conn.rollback()

In [145]:
%load_ext autoreload
%autoreload 2
from sql_queries import *

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Songplay table

In [146]:
# drop table
cur.execute(songplay_table_drop)
conn.commit()

# create table
cur.execute(songplay_table_create)
conn.commit()

# load data into clusters
cur.execute(songplay_table_insert)
conn.commit()

In [147]:
cur.execute("select * from songplays")
songplay_df = pd.DataFrame(cur.fetchall(), columns=['pk', 'ts', 'user_id', 'level', 'song_id',
                                     'artist_id', 'session_id', 'location', 'user_agent'])
songplay_df

,pk,ts,user_id,level,song_id,artist_id,session_id,location,user_agent
0,32,2018-11-05 01:48:00.796,44,paid,SOOXLKF12A6D4F594A,ARF5M7Q1187FB501E8,237,"Waterloo-Cedar Falls, IA",Mozilla/5.0 (Macintosh; Intel Mac OS X 10.9; r...
1,96,2018-11-28 08:09:40.796,58,paid,SOEHGER12A8AE47DB2,ART7Q0O1187B9A79FA,887,"Augusta-Richmond County, GA-SC","""Mozilla/5.0 (Windows NT 6.3; WOW64) AppleWebK..."
2,160,2018-11-06 20:59:02.796,97,paid,SOSCHDW12A8C130FD7,AR40GSU1187FB3AA01,293,"Lansing-East Lansing, MI","""Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/5..."
3,224,2018-11-23 21:42:09.796,85,paid,SONEGWF12A8C133022,ARI75KH1187FB4C77C,891,"Red Bluff, CA","""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_..."
4,288,2018-11-15 17:09:53.796,42,paid,SOFDXZL12A6D4F96AB,ARCZODZ1187B9B92E9,404,"New York-Newark-Jersey City, NY-NJ-PA","""Mozilla/5.0 (Windows NT 6.3; WOW64) AppleWebK..."
5,352,2018-11-04 06:51:12.796,25,paid,SORKKTY12A8C132F3E,ARIH5GU1187FB4C958,128,"Marinette, WI-MI","""Mozilla/5.0 (Windows NT 6.3; WOW64) AppleWebK..."
6,18,2018-11-12 17:26:37.796,73,paid,SOCJWPI12A8C13D357,ARRB3GQ1187FB52D14,294,"Tampa-St. Petersburg-Clearwater, FL","""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4..."
7,82,2018-11-04 07:31:31.796,25,paid,SOHWVJJ12AB0185F6D,ARASYMJ1187B9ACAF2,128,"Marinette, WI-MI","""Mozilla/5.0 (Windows NT 6.3; WOW64) AppleWebK..."
8,146,2018-11-22 17:18:20.796,83,free,SOSJEEG12A6BD5571F,AR8RQVM1187B9A2645,772,"Lubbock, TX","""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4..."
9,210,2018-11-13 11:46:10.796,50,free,SOBJDDA12A6BD53159,ARCS4GZ1187FB469EB,430,"New Haven-Milford, CT","""Mozilla/5.0 (Windows NT 6.3; WOW64) AppleWebK..."


In [148]:
songplay_df.shape[0]

333

In [149]:
conn.close()